In [ ]:
# 1) Drive bağla
from google.colab import drive  # type: ignore
drive.mount('/content/drive')

In [ ]:
# 🔍 TANI: Extraction nereye yazıyor? RAR içeriği ne?
from pathlib import Path
import subprocess
import os

print("="*60)
print("🔍 EXTRACTION TANI RAPORU")
print("="*60)

# 1) RAR dosyası hala var mı ve boyutu ne?
rar_path = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn.rar')
print(f"\n📦 RAR Dosyası: {rar_path}")
print(f"   Mevcut: {rar_path.exists()}")
if rar_path.exists():
    print(f"   Boyut: {rar_path.stat().st_size / (1024**3):.1f} GB")

# 2) unrar arşiv içeriğini listele (ilk 30 dosya)
print("\n📋 RAR İçeriği (ilk 30 dosya):")
try:
    result = subprocess.run(
        f'unrar lb "{rar_path}" | head -30',
        shell=True, capture_output=True, text=True, timeout=60
    )
    print(result.stdout if result.stdout else "(boş)")
    if result.stderr:
        print("STDERR:", result.stderr[:200])
except Exception as e:
    print(f"Hata: {e}")

# 3) Olası extraction hedefleri kontrol et
print("\n📁 Olası Hedef Dizinler:")
possible_dirs = [
    Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn'),
    Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/ds2_complete_tmn'),
    Path('/content/ds2_complete_tmn'),
    Path('/content/drive/MyDrive/omr_dataset/dataset/ds2'),
]

for d in possible_dirs:
    if d.exists():
        # Boyut hesapla
        try:
            result = subprocess.run(f"du -sh '{d}' 2>/dev/null", shell=True, capture_output=True, text=True, timeout=30)
            size = result.stdout.strip().split()[0] if result.stdout else "?"
        except:
            size = "?"
        
        # Dosya sayısı
        try:
            files = list(d.rglob('*'))
            file_count = len([f for f in files if f.is_file()])
        except:
            file_count = "?"
        
        print(f"   ✅ {d}")
        print(f"      Boyut: {size}, Dosya sayısı: {file_count}")
        
        # Alt klasörler
        try:
            subdirs = [x.name for x in d.iterdir() if x.is_dir()][:10]
            if subdirs:
                print(f"      Alt klasörler: {subdirs}")
        except:
            pass
    else:
        print(f"   ❌ {d} (yok)")

# 4) /content altında ne var?
print("\n📁 /content altındaki büyük öğeler:")
subprocess.run("du -sh /content/*/ 2>/dev/null | sort -rh | head -10", shell=True)

print("\n" + "="*60)
print("💡 Eğer dosyalar farklı bir yere çıkıyorsa, extraction")
print("   notebook'undaki EXTRACT_DIR'i güncelle!")
print("="*60)

In [ ]:
# 2) İlerleme takibi - bu hücreyi tekrar tekrar çalıştır!
from pathlib import Path
import time

# Ana dizin
BASE = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')

# Takip edilecek klasörler
FOLDERS = ['images', 'segmentation', 'instance']

def count_folder(folder_path: Path):
    """Dosya sayısı ve toplam boyut hesapla"""
    if not folder_path.exists():
        return 0, 0
    files = list(folder_path.rglob('*'))
    files = [f for f in files if f.is_file()]
    count = len(files)
    size = sum(f.stat().st_size for f in files)
    return count, size

def format_size(bytes_size):
    """Boyutu okunabilir formata çevir"""
    if bytes_size < 1024:
        return f"{bytes_size} B"
    elif bytes_size < 1024**2:
        return f"{bytes_size/1024:.1f} KB"
    elif bytes_size < 1024**3:
        return f"{bytes_size/1024**2:.1f} MB"
    else:
        return f"{bytes_size/1024**3:.2f} GB"

# Sonuçları göster
print(f"{'='*60}")
print(f"📊 EXTRACTION İLERLEME RAPORU - {time.strftime('%H:%M:%S')}")
print(f"{'='*60}")
print(f"Base: {BASE}\n")

total_files = 0
total_size = 0

for folder in FOLDERS:
    folder_path = BASE / folder
    count, size = count_folder(folder_path)
    total_files += count
    total_size += size
    
    status = "✅" if folder_path.exists() else "❌"
    print(f"{status} {folder:15} | Dosya: {count:>8,} | Boyut: {format_size(size):>10}")

print(f"{'─'*60}")
print(f"📦 TOPLAM           | Dosya: {total_files:>8,} | Boyut: {format_size(total_size):>10}")
print(f"{'='*60}")
print(f"\n💡 Bu hücreyi tekrar çalıştırarak ilerlemeyi takip et!")

In [ ]:
# 3) Otomatik yenileme (opsiyonel) - 30 saniyede bir güncelle
from pathlib import Path
import time
from IPython.display import clear_output

BASE = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')
FOLDERS = ['images', 'segmentation', 'instance']

def count_folder(folder_path: Path):
    if not folder_path.exists():
        return 0, 0
    files = [f for f in folder_path.rglob('*') if f.is_file()]
    return len(files), sum(f.stat().st_size for f in files)

def format_size(b):
    if b < 1024**3: return f"{b/1024**2:.1f} MB"
    return f"{b/1024**3:.2f} GB"

# Kaç kez güncellensin? (30 saniye arayla)
ITERATIONS = 20  # 20 x 30s = 10 dakika

for i in range(ITERATIONS):
    clear_output(wait=True)
    print(f"{'='*60}")
    print(f"📊 CANLI TAKİP [{i+1}/{ITERATIONS}] - {time.strftime('%H:%M:%S')}")
    print(f"{'='*60}")
    
    total_files, total_size = 0, 0
    for folder in FOLDERS:
        folder_path = BASE / folder
        count, size = count_folder(folder_path)
        total_files += count
        total_size += size
        status = "✅" if folder_path.exists() else "❌"
        print(f"{status} {folder:15} | Dosya: {count:>8,} | Boyut: {format_size(size):>10}")
    
    print(f"{'─'*60}")
    print(f"📦 TOPLAM           | Dosya: {total_files:>8,} | Boyut: {format_size(total_size):>10}")
    print(f"\n⏳ Sonraki güncelleme 30 saniye sonra... (Durdurmak için Interrupt)")
    
    if i < ITERATIONS - 1:
        time.sleep(30)

print("\n✅ Takip tamamlandı!")